In [14]:
import numpy as np
import pandas as pd
import re

In [11]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [15]:
df = pd.read_csv('gurgaon_properties_cleaned_v1.csv')

In [16]:
df.duplicated().sum()

122

In [3]:
df.head(1)
# focus is on -> areaWithType, additionalRoom, agePossession, furnishDetails, features 

property_type                  society     sector  price  price_per_sqft  \
0          flat  signature global park 4  sector 36   0.82          7585.0   

     area                                       areaWithType  bedRoom  \
0  1081.0  Super Built up area 1081(100.43 sq.m.)Carpet a...        3   

   bathroom balcony additionalRoom  floorNum facing    agePossession  \
0         2       2  not available       2.0    NaN  0 to 1 Year Old   

                                     nearbyLocations furnishDetails  \
0  ['Sector 55-56 Metro Station', 'The Leaf Mall ...            NaN   

              features  
0  ['Lift(s)', 'Park']

## 1. areaWithType

In [4]:
df.sample(5)[['price','area','areaWithType']]

price    area                                       areaWithType
973    0.75  1081.0                 Built Up area: 1081 (100.43 sq.m.)
953    1.70  2200.0  Built Up area: 2200 (204.39 sq.m.)Carpet area:...
3238   0.78  1350.0             Super Built up area 1350(125.42 sq.m.)
3413   1.54  2364.0  Super Built up area 2364(219.62 sq.m.)Built Up...
1116   2.35  1650.0             Super Built up area 1650(153.29 sq.m.)

In [5]:
# This function extracts the Super Built up area
def get_super_built_up_area(text):
    match = re.search(r'Super Built up area (\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [6]:
# This function extracts the Built Up area or Carpet area
def get_area(text, area_type):
    match = re.search(area_type + r'\s*:\s*(\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [7]:
# This function checks if the area is provided in sq.m. and converts it to sqft if needed
def convert_to_sqft(text, area_value):
    if area_value is None:
        return None
    match = re.search(r'{} \((\d+\.?\d*) sq.m.\)'.format(area_value), text)
    if match:
        sq_m_value = float(match.group(1))
        return sq_m_value * 10.7639  # conversion factor from sq.m. to sqft
    return area_value

In [8]:
# Extract Super Built up area and convert to sqft if needed
df['super_built_up_area'] = df['areaWithType'].apply(get_super_built_up_area)
df['super_built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['super_built_up_area']), axis=1)

# Extract Built Up area and convert to sqft if needed
df['built_up_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Built Up area'))
df['built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['built_up_area']), axis=1)

# Extract Carpet area and convert to sqft if needed
df['carpet_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Carpet area'))
df['carpet_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['carpet_area']), axis=1)

In [9]:
df[['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].sample(5)

price property_type      area  \
386    5.50         house    1000.0   
1435   1.05          flat    1822.0   
290    1.00          flat    1110.0   
2877   0.85          flat     685.0   
1638   0.51          flat  566667.0   

                                           areaWithType  super_built_up_area  \
386                          Plot area 1000(92.9 sq.m.)                  NaN   
1435  Super Built up area 1822(169.27 sq.m.)Built Up...               1822.0   
290                    Carpet area: 1110 (103.12 sq.m.)                  NaN   
2877                     Carpet area: 685 (63.64 sq.m.)                  NaN   
1638               Carpet area: 514396 (47788.95 sq.m.)                  NaN   

      built_up_area  carpet_area  
386             NaN          NaN  
1435         1600.0       1400.0  
290             NaN       1110.0  
2877            NaN        685.0  
1638            NaN     514396.0

In [10]:
df.duplicated().sum()

122

In [577]:
df[~((df['super_built_up_area'].isnull()) | (df['built_up_area'].isnull()) | (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].shape

(534, 7)

In [579]:
df[df['areaWithType'].str.contains('Plot')][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].head(5)

price property_type    area  \
9     1.2         house  1186.0   
10    4.5         house  1350.0   
15    1.2         house   846.0   
19    5.5         house  1440.0   
25    9.0         house  3240.0   

                                                       areaWithType  \
9                                   Plot area 1185.51(110.14 sq.m.)   
10                                      Plot area 150(125.42 sq.m.)   
15  Plot area 846(78.6 sq.m.)Built Up area: 846 sq.ft. (78.6 sq.m.)   
19                                      Plot area 160(133.78 sq.m.)   
25                                      Plot area 360(301.01 sq.m.)   

    super_built_up_area  built_up_area  carpet_area  
9                   NaN            NaN          NaN  
10                  NaN            NaN          NaN  
15                  NaN          846.0          NaN  
19                  NaN            NaN          NaN  
25                  NaN            NaN          NaN

In [580]:
df.isnull().sum()

property_type             0
society                   1
sector                    0
price                    18
price_per_sqft           18
area                     18
areaWithType              0
bedRoom                   0
bathroom                  0
balcony                   0
additionalRoom            0
floorNum                 19
facing                 1105
agePossession             1
nearbyLocations         177
furnishDetails          981
features                635
super_built_up_area    1888
built_up_area          2616
carpet_area            1859
dtype: int64

In [581]:
all_nan_df = df[((df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']]

In [582]:
all_nan_df.head()

price property_type    area                     areaWithType  \
9     1.2         house  1186.0  Plot area 1185.51(110.14 sq.m.)   
10    4.5         house  1350.0      Plot area 150(125.42 sq.m.)   
19    5.5         house  1440.0      Plot area 160(133.78 sq.m.)   
25    9.0         house  3240.0      Plot area 360(301.01 sq.m.)   
26    7.0         house  2475.0      Plot area 275(229.94 sq.m.)   

    super_built_up_area  built_up_area  carpet_area  
9                   NaN            NaN          NaN  
10                  NaN            NaN          NaN  
19                  NaN            NaN          NaN  
25                  NaN            NaN          NaN  
26                  NaN            NaN          NaN

In [583]:
all_nan_index = df[((df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].index

In [585]:
# Function to extract plot area from 'areaWithType' column
def extract_plot_area(area_with_type):
    match = re.search(r'Plot area (\d+\.?\d*)', area_with_type)
    return float(match.group(1)) if match else None

In [586]:
all_nan_df['built_up_area'] = all_nan_df['areaWithType'].apply(extract_plot_area)

# Update the original dataframe
#gurgaon_properties.update(filtered_rows)

In [589]:
all_nan_df

price property_type      area                     areaWithType  \
9      1.20         house    1186.0  Plot area 1185.51(110.14 sq.m.)   
10     4.50         house    1350.0      Plot area 150(125.42 sq.m.)   
19     5.50         house    1440.0      Plot area 160(133.78 sq.m.)   
25     9.00         house    3240.0      Plot area 360(301.01 sq.m.)   
26     7.00         house    2475.0      Plot area 275(229.94 sq.m.)   
28     0.40         house     450.0        Plot area 50(41.81 sq.m.)   
29     5.65         house    5400.0      Plot area 600(501.68 sq.m.)   
31     4.40         house    2880.0      Plot area 320(267.56 sq.m.)   
38      NaN         house       NaN      Plot area 350(292.64 sq.m.)   
40     6.10         house    2430.0      Plot area 270(225.75 sq.m.)   
45     5.50         house    1800.0      Plot area 200(167.23 sq.m.)   
48     1.25         house   82781.0     Plot area 115(7692.86 sq.m.)   
66     0.50         house      55.0         Plot area 55(5.11 sq.m.)   
67     0.67         house     783.0        Plot area 87(72.74 sq.m.)   
70     4.15         house    2250.0      Plot area 250(209.03 sq.m.)   
74     7.60         house    1961.0      Plot area 215(179.77 sq.m.)   
77     2.99         house    1449.0      Plot area 161(134.62 sq.m.)   
81     1.50         house    1980.0      Plot area 220(183.95 sq.m.)   
96    10.45         house    3240.0      Plot area 360(301.01 sq.m.)   
119    7.49         house    2700.0      Plot area 300(250.84 sq.m.)   
126    3.20         house    2250.0      Plot area 250(209.03 sq.m.)   
127    6.95         house    1800.0      Plot area 192(160.54 sq.m.)   
136    1.95         house     920.0       Plot area 920(85.47 sq.m.)   
139    4.70         house    2160.0      Plot area 240(200.67 sq.m.)   
140    0.70         house     864.0       Plot area 864(80.27 sq.m.)   
142    6.50         house    5400.0     Plot area 5400(501.68 sq.m.)   
149    3.90         house    1836.0      Plot area 204(170.57 sq.m.)   
155    2.25         house     900.0       Plot area 100(83.61 sq.m.)   
156   11.00         house    4125.0      Plot area 500(418.06 sq.m.)   
162    4.35         house    3222.0      Plot area 358(299.33 sq.m.)   
183    6.75         house    2430.0      Plot area 270(225.75 sq.m.)   
186    1.50         house    1300.0     Plot area 1300(120.77 sq.m.)   
192    8.50         house    4770.0      Plot area 530(443.15 sq.m.)   
239    3.15         house    1548.0      Plot area 172(143.81 sq.m.)   
247    0.89         house     477.0       Plot area 477(44.31 sq.m.)   
255    3.75         house    1467.0      Plot area 163(136.29 sq.m.)   
273    3.65         house    3450.0     Plot area 3450(320.52 sq.m.)   
276    7.50         house    3240.0      Plot area 360(301.01 sq.m.)   
278    5.50         house    2592.0       Plot area 288(240.8 sq.m.)   
304     NaN         house       NaN      Plot area 670(560.21 sq.m.)   
309    5.00         house    2367.0       Plot area 263(219.9 sq.m.)   
317    5.80         house    4518.0      Plot area 502(419.74 sq.m.)   
324    6.50         house    3150.0      Plot area 350(292.64 sq.m.)   
329    2.70         house    1450.0     Plot area 1450(134.71 sq.m.)   
333    4.00         house    2250.0      Plot area 250(209.03 sq.m.)   
334    0.75         house    1170.0       Plot area 130(108.7 sq.m.)   
342    7.10         house     251.0        Plot area 240(22.3 sq.m.)   
350    8.50         house    3240.0      Plot area 360(301.01 sq.m.)   
354    6.50         house    2367.0       Plot area 263(219.9 sq.m.)   
355    0.95         house    1070.0      Plot area 1070(99.41 sq.m.)   
372    4.50         house    1800.0      Plot area 200(167.23 sq.m.)   
386    5.50         house    1000.0       Plot area 1000(92.9 sq.m.)   
392    6.50         house    2500.0     Plot area 2500(232.26 sq.m.)   
393    1.00         house    1215.0      Plot area 135(112.88 sq.m.)   
397    6.00         house    2430.0      Plot 

In [590]:
def convert_scale(row):
    if np.isnan(row['area']) or np.isnan(row['built_up_area']):
        return row['built_up_area']
    else:
        if round(row['area']/row['built_up_area']) == 9.0:
            return row['built_up_area'] * 9
        elif round(row['area']/row['built_up_area']) == 11.0:
            return row['built_up_area'] * 10.7
        else:
            return row['built_up_area']

In [591]:
all_nan_df['built_up_area'] = all_nan_df.apply(convert_scale,axis=1)

In [592]:
all_nan_df

price property_type      area                     areaWithType  \
9      1.20         house    1186.0  Plot area 1185.51(110.14 sq.m.)   
10     4.50         house    1350.0      Plot area 150(125.42 sq.m.)   
19     5.50         house    1440.0      Plot area 160(133.78 sq.m.)   
25     9.00         house    3240.0      Plot area 360(301.01 sq.m.)   
26     7.00         house    2475.0      Plot area 275(229.94 sq.m.)   
28     0.40         house     450.0        Plot area 50(41.81 sq.m.)   
29     5.65         house    5400.0      Plot area 600(501.68 sq.m.)   
31     4.40         house    2880.0      Plot area 320(267.56 sq.m.)   
38      NaN         house       NaN      Plot area 350(292.64 sq.m.)   
40     6.10         house    2430.0      Plot area 270(225.75 sq.m.)   
45     5.50         house    1800.0      Plot area 200(167.23 sq.m.)   
48     1.25         house   82781.0     Plot area 115(7692.86 sq.m.)   
66     0.50         house      55.0         Plot area 55(5.11 sq.m.)   
67     0.67         house     783.0        Plot area 87(72.74 sq.m.)   
70     4.15         house    2250.0      Plot area 250(209.03 sq.m.)   
74     7.60         house    1961.0      Plot area 215(179.77 sq.m.)   
77     2.99         house    1449.0      Plot area 161(134.62 sq.m.)   
81     1.50         house    1980.0      Plot area 220(183.95 sq.m.)   
96    10.45         house    3240.0      Plot area 360(301.01 sq.m.)   
119    7.49         house    2700.0      Plot area 300(250.84 sq.m.)   
126    3.20         house    2250.0      Plot area 250(209.03 sq.m.)   
127    6.95         house    1800.0      Plot area 192(160.54 sq.m.)   
136    1.95         house     920.0       Plot area 920(85.47 sq.m.)   
139    4.70         house    2160.0      Plot area 240(200.67 sq.m.)   
140    0.70         house     864.0       Plot area 864(80.27 sq.m.)   
142    6.50         house    5400.0     Plot area 5400(501.68 sq.m.)   
149    3.90         house    1836.0      Plot area 204(170.57 sq.m.)   
155    2.25         house     900.0       Plot area 100(83.61 sq.m.)   
156   11.00         house    4125.0      Plot area 500(418.06 sq.m.)   
162    4.35         house    3222.0      Plot area 358(299.33 sq.m.)   
183    6.75         house    2430.0      Plot area 270(225.75 sq.m.)   
186    1.50         house    1300.0     Plot area 1300(120.77 sq.m.)   
192    8.50         house    4770.0      Plot area 530(443.15 sq.m.)   
239    3.15         house    1548.0      Plot area 172(143.81 sq.m.)   
247    0.89         house     477.0       Plot area 477(44.31 sq.m.)   
255    3.75         house    1467.0      Plot area 163(136.29 sq.m.)   
273    3.65         house    3450.0     Plot area 3450(320.52 sq.m.)   
276    7.50         house    3240.0      Plot area 360(301.01 sq.m.)   
278    5.50         house    2592.0       Plot area 288(240.8 sq.m.)   
304     NaN         house       NaN      Plot area 670(560.21 sq.m.)   
309    5.00         house    2367.0       Plot area 263(219.9 sq.m.)   
317    5.80         house    4518.0      Plot area 502(419.74 sq.m.)   
324    6.50         house    3150.0      Plot area 350(292.64 sq.m.)   
329    2.70         house    1450.0     Plot area 1450(134.71 sq.m.)   
333    4.00         house    2250.0      Plot area 250(209.03 sq.m.)   
334    0.75         house    1170.0       Plot area 130(108.7 sq.m.)   
342    7.10         house     251.0        Plot area 240(22.3 sq.m.)   
350    8.50         house    3240.0      Plot area 360(301.01 sq.m.)   
354    6.50         house    2367.0       Plot area 263(219.9 sq.m.)   
355    0.95         house    1070.0      Plot area 1070(99.41 sq.m.)   
372    4.50         house    1800.0      Plot area 200(167.23 sq.m.)   
386    5.50         house    1000.0       Plot area 1000(92.9 sq.m.)   
392    6.50         house    2500.0     Plot area 2500(232.26 sq.m.)   
393    1.00         house    1215.0      Plot area 135(112.88 sq.m.)   
397    6.00         house    2430.0      Plot 

In [593]:
# update the original dataframe
df.update(all_nan_df)

In [594]:
df.isnull().sum()

property_type             0
society                   1
sector                    0
price                    18
price_per_sqft           18
area                     18
areaWithType              0
bedRoom                   0
bathroom                  0
balcony                   0
additionalRoom            0
floorNum                 19
facing                 1105
agePossession             1
nearbyLocations         177
furnishDetails          981
features                635
super_built_up_area    1888
built_up_area          2070
carpet_area            1859
dtype: int64

In [595]:
df.head()

property_type                      society      sector  price  \
0          flat      signature global park 4   sector 36   0.82   
1          flat             smart world gems   sector 89   0.95   
2          flat                pyramid elite   sector 86   0.46   
3          flat       breez global hill view  sohna road   0.32   
4          flat  bestech park view sanskruti   sector 92   1.60   

   price_per_sqft     area  \
0          7585.0   1081.0   
1          8600.0   1105.0   
2            79.0  58228.0   
3          5470.0    585.0   
4          8020.0   1995.0   

                                                                                                             areaWithType  \
0                                             Super Built up area 1081(100.43 sq.m.)Carpet area: 650 sq.ft. (60.39 sq.m.)   
1                                                                                        Carpet area: 1103 (102.47 sq.m.)   
2                                                                                      Carpet area: 58141 (5401.48 sq.m.)   
3                                                   Built Up area: 1000 (92.9 sq.m.)Carpet area: 585 sq.ft. (54.35 sq.m.)   
4  Super Built up area 1995(185.34 sq.m.)Built Up area: 1615 sq.ft. (150.04 sq.m.)Carpet area: 1476 sq.ft. (137.12 sq.m.)   

   bedRoom  bathroom balcony           additionalRoom  floorNum      facing  \
0        3         2       2            not available       2.0         NaN   
1        2         2       2  study room,servant room       4.0         NaN   
2        2         2       1            not available       0.0         NaN   
3        2         2       1            not available      17.0         NaN   
4        3         4      3+      servant room,others      10.0  North-West   

        agePossession  \
0     0 to 1 Year Old   
1     Within 6 months   
2  Under Construction   
3     0 to 1 Year Old   
4     1 to 5 Year Old   

                                                                                                                                                                                                                                                                                                                                                nearbyLocations  \
0                                                    ['Sector 55-56 Metro Station', 'The Leaf Mall Sohna', 'Omaxe Gurgaon Mall', 'Badshahpur Sohna Rd Hwy', 'Vidya Niketan School', 'KIIT College of Engg', 'Scottish High International School', 'Rawal Institutions', 'Park Hospital', 'Indira Gandhi Intl Airport', 'Spaze Business Park', 'DLF Cyber City']   
1  ['Sant Soordas Sihi Metro Station', 'Airia Mall', 'Dwarka Expressway', 'Pataudi Road', 'NH-8', 'KMP Expressway', 'RPS International School', 'Genesis Hospital Sector 84', 'Gurgaon Railway Station', 'Vatika Business Park Sector 49', 'Holiday Inn Gurugram Sector 90', 'De Adventure Amusement Park', 'National Tennis Academy Sector 98', 'IMT Manesar']   
2                                                                                                                                                ['V3S Sapphire Ninety Mall', 'Sector 86 Road', 'Dwarka Expy', 'RPS International School', 'SGT University', 'Shree Radhey Hospital And Trauma Center', 'Indira Gandhi Intl Airport', 'Garhi Harsaru Junction']   
3                       ['Huda City Centre', 'Omaxe City Centre', 'Ninex City Mart', 'Vaels Ravindra Bharathi Global School', 'Anjali nursery', 'Polaris Hospital', 'Axis Bank', 'Triangular Park', 'Flow Sports Life Badminton Club', 'Basai Dhankot Railway Station', 'Cars24', 'Radisson Hotel Gurugram', "McDonald's", 'Petrol Pump', 'Indian Post Office']   
4                                   ['Sapphire 93 Mall', 'Dwarka Expressway', 'NH-8, Imt Manesar', 'Western Peripheral Expressway', 'RPS International School Sector 89', 'DPG Institute of Technology', 'Aarvy Healthcare Hospital', 'Indira Gandhi International Airport

## 2.additionalRoom

In [596]:
df['additionalRoom'].value_counts()

not available                                    1587
servant room                                      705
study room                                        250
others                                            225
pooja room                                        165
store room                                         99
study room,servant room                            99
pooja room,servant room                            82
pooja room,study room,servant room,store room      72
servant room,others                                60
pooja room,study room,servant room                 55
pooja room,study room,servant room,others          54
servant room,pooja room                            38
servant room,store room                            33
study room,others                                  29
pooja room,study room                              22
pooja room,others                                  17
pooja room,store room                              15
pooja room,store room,study 

In [597]:
# additional room
# List of new columns to be created
new_cols = ['study room', 'servant room', 'store room', 'pooja room', 'others']

# Populate the new columns based on the "additionalRoom" column
for col in new_cols:
    df[col] = df['additionalRoom'].str.contains(col).astype(int)

In [598]:
df.sample(5)[['additionalRoom','study room', 'servant room', 'store room', 'pooja room', 'others']]

additionalRoom  study room  servant room  store room  \
1456               study room           1             0           0   
79               servant room           0             1           0   
870             not available           0             0           0   
2520            not available           0             0           0   
2776  pooja room,servant room           0             1           0   

      pooja room  others  
1456           0       0  
79             0       0  
870            0       0  
2520           0       0  
2776           1       0

In [599]:
df.head()

property_type                      society      sector  price  \
0          flat      signature global park 4   sector 36   0.82   
1          flat             smart world gems   sector 89   0.95   
2          flat                pyramid elite   sector 86   0.46   
3          flat       breez global hill view  sohna road   0.32   
4          flat  bestech park view sanskruti   sector 92   1.60   

   price_per_sqft     area  \
0          7585.0   1081.0   
1          8600.0   1105.0   
2            79.0  58228.0   
3          5470.0    585.0   
4          8020.0   1995.0   

                                                                                                             areaWithType  \
0                                             Super Built up area 1081(100.43 sq.m.)Carpet area: 650 sq.ft. (60.39 sq.m.)   
1                                                                                        Carpet area: 1103 (102.47 sq.m.)   
2                                                                                      Carpet area: 58141 (5401.48 sq.m.)   
3                                                   Built Up area: 1000 (92.9 sq.m.)Carpet area: 585 sq.ft. (54.35 sq.m.)   
4  Super Built up area 1995(185.34 sq.m.)Built Up area: 1615 sq.ft. (150.04 sq.m.)Carpet area: 1476 sq.ft. (137.12 sq.m.)   

   bedRoom  bathroom balcony           additionalRoom  floorNum      facing  \
0        3         2       2            not available       2.0         NaN   
1        2         2       2  study room,servant room       4.0         NaN   
2        2         2       1            not available       0.0         NaN   
3        2         2       1            not available      17.0         NaN   
4        3         4      3+      servant room,others      10.0  North-West   

        agePossession  \
0     0 to 1 Year Old   
1     Within 6 months   
2  Under Construction   
3     0 to 1 Year Old   
4     1 to 5 Year Old   

                                                                                                                                                                                                                                                                                                                                                nearbyLocations  \
0                                                    ['Sector 55-56 Metro Station', 'The Leaf Mall Sohna', 'Omaxe Gurgaon Mall', 'Badshahpur Sohna Rd Hwy', 'Vidya Niketan School', 'KIIT College of Engg', 'Scottish High International School', 'Rawal Institutions', 'Park Hospital', 'Indira Gandhi Intl Airport', 'Spaze Business Park', 'DLF Cyber City']   
1  ['Sant Soordas Sihi Metro Station', 'Airia Mall', 'Dwarka Expressway', 'Pataudi Road', 'NH-8', 'KMP Expressway', 'RPS International School', 'Genesis Hospital Sector 84', 'Gurgaon Railway Station', 'Vatika Business Park Sector 49', 'Holiday Inn Gurugram Sector 90', 'De Adventure Amusement Park', 'National Tennis Academy Sector 98', 'IMT Manesar']   
2                                                                                                                                                ['V3S Sapphire Ninety Mall', 'Sector 86 Road', 'Dwarka Expy', 'RPS International School', 'SGT University', 'Shree Radhey Hospital And Trauma Center', 'Indira Gandhi Intl Airport', 'Garhi Harsaru Junction']   
3                       ['Huda City Centre', 'Omaxe City Centre', 'Ninex City Mart', 'Vaels Ravindra Bharathi Global School', 'Anjali nursery', 'Polaris Hospital', 'Axis Bank', 'Triangular Park', 'Flow Sports Life Badminton Club', 'Basai Dhankot Railway Station', 'Cars24', 'Radisson Hotel Gurugram', "McDonald's", 'Petrol Pump', 'Indian Post Office']   
4                                   ['Sapphire 93 Mall', 'Dwarka Expressway', 'NH-8, Imt Manesar', 'Western Peripheral Expressway', 'RPS International School Sector 89', 'DPG Institute of Technology', 'Aarvy Healthcare Hospital', 'Indira Gandhi International Airport

## 3.agePossession

In [600]:
df['agePossession'].value_counts()

1 to 5 Year Old       1676
5 to 10 Year Old       575
0 to 1 Year Old        530
undefined              332
10+ Year Old           310
Under Construction      90
Within 6 months         70
Within 3 months         26
Dec 2023                22
By 2023                 19
By 2024                 17
Dec 2024                16
Mar 2024                14
Oct 2024                 8
Jan 2024                 8
Aug 2023                 7
Dec 2025                 7
Jun 2024                 7
Nov 2023                 5
Aug 2024                 4
Sep 2023                 4
Jul 2024                 4
Oct 2023                 4
By 2025                  4
Nov 2024                 3
May 2024                 3
Feb 2024                 3
Jan 2025                 3
By 2027                  2
Sep 2025                 2
Dec 2026                 2
Jul 2025                 2
Jan 2026                 2
Jun 2027                 2
Jul 2027                 2
Mar 2025                 2
Oct 2025                 2
A

In [601]:
def categorize_age_possession(value):
    if pd.isna(value):
        return "Undefined"
    if "0 to 1 Year Old" in value or "Within 6 months" in value or "Within 3 months" in value:
        return "New Property"
    if "1 to 5 Year Old" in value:
        return "Relatively New"
    if "5 to 10 Year Old" in value:
        return "Moderately Old"
    if "10+ Year Old" in value:
        return "Old Property"
    if "Under Construction" in value or "By" in value:
        return "Under Construction"
    try:
        # For entries like 'May 2024'
        int(value.split(" ")[-1])
        return "Under Construction"
    except:
        return "Undefined"

In [602]:
df['agePossession'] = df['agePossession'].apply(categorize_age_possession)

In [603]:
df['agePossession'].value_counts()

Relatively New        1676
New Property           626
Moderately Old         575
Undefined              333
Old Property           310
Under Construction     283
Name: agePossession, dtype: int64

In [604]:
df.head()

property_type                      society      sector  price  \
0          flat      signature global park 4   sector 36   0.82   
1          flat             smart world gems   sector 89   0.95   
2          flat                pyramid elite   sector 86   0.46   
3          flat       breez global hill view  sohna road   0.32   
4          flat  bestech park view sanskruti   sector 92   1.60   

   price_per_sqft     area  \
0          7585.0   1081.0   
1          8600.0   1105.0   
2            79.0  58228.0   
3          5470.0    585.0   
4          8020.0   1995.0   

                                                                                                             areaWithType  \
0                                             Super Built up area 1081(100.43 sq.m.)Carpet area: 650 sq.ft. (60.39 sq.m.)   
1                                                                                        Carpet area: 1103 (102.47 sq.m.)   
2                                                                                      Carpet area: 58141 (5401.48 sq.m.)   
3                                                   Built Up area: 1000 (92.9 sq.m.)Carpet area: 585 sq.ft. (54.35 sq.m.)   
4  Super Built up area 1995(185.34 sq.m.)Built Up area: 1615 sq.ft. (150.04 sq.m.)Carpet area: 1476 sq.ft. (137.12 sq.m.)   

   bedRoom  bathroom balcony           additionalRoom  floorNum      facing  \
0        3         2       2            not available       2.0         NaN   
1        2         2       2  study room,servant room       4.0         NaN   
2        2         2       1            not available       0.0         NaN   
3        2         2       1            not available      17.0         NaN   
4        3         4      3+      servant room,others      10.0  North-West   

        agePossession  \
0        New Property   
1        New Property   
2  Under Construction   
3        New Property   
4      Relatively New   

                                                                                                                                                                                                                                                                                                                                                nearbyLocations  \
0                                                    ['Sector 55-56 Metro Station', 'The Leaf Mall Sohna', 'Omaxe Gurgaon Mall', 'Badshahpur Sohna Rd Hwy', 'Vidya Niketan School', 'KIIT College of Engg', 'Scottish High International School', 'Rawal Institutions', 'Park Hospital', 'Indira Gandhi Intl Airport', 'Spaze Business Park', 'DLF Cyber City']   
1  ['Sant Soordas Sihi Metro Station', 'Airia Mall', 'Dwarka Expressway', 'Pataudi Road', 'NH-8', 'KMP Expressway', 'RPS International School', 'Genesis Hospital Sector 84', 'Gurgaon Railway Station', 'Vatika Business Park Sector 49', 'Holiday Inn Gurugram Sector 90', 'De Adventure Amusement Park', 'National Tennis Academy Sector 98', 'IMT Manesar']   
2                                                                                                                                                ['V3S Sapphire Ninety Mall', 'Sector 86 Road', 'Dwarka Expy', 'RPS International School', 'SGT University', 'Shree Radhey Hospital And Trauma Center', 'Indira Gandhi Intl Airport', 'Garhi Harsaru Junction']   
3                       ['Huda City Centre', 'Omaxe City Centre', 'Ninex City Mart', 'Vaels Ravindra Bharathi Global School', 'Anjali nursery', 'Polaris Hospital', 'Axis Bank', 'Triangular Park', 'Flow Sports Life Badminton Club', 'Basai Dhankot Railway Station', 'Cars24', 'Radisson Hotel Gurugram', "McDonald's", 'Petrol Pump', 'Indian Post Office']   
4                                   ['Sapphire 93 Mall', 'Dwarka Expressway', 'NH-8, Imt Manesar', 'Western Peripheral Expressway', 'RPS International School Sector 89', 'DPG Institute of Technology', 'Aarvy Healthcare Hospital', 'Indira Gandhi International Airport

## 4.furnishDetails

In [605]:
df.sample(5)[['furnishDetails','features']]

furnishDetails  \
1623                                                                                                                                                                                                                                                                   NaN   
2713       ['3 Fan', '1 Exhaust Fan', '3 Geyser', '7 Light', '1 Chimney', '2 Wardrobe', 'No AC', 'No Bed', 'No Curtains', 'No Dining Table', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']   
130     ['7 Wardrobe', '17 Light', '5 AC', '1 Modular Kitchen', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']   
1419                                                                                                                                                                                                                                                                   NaN   
2213  ['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']   

                                                                                                                                                                                                                                                                         features  
1623  ['Security / Fire Alarm', 'Power Back-up', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'No open drainage around', 'Visitor Parking', 'Park', 'Shopping Centre', 'Fitness Centre / GYM', 'Rain Water Harvesting', 'Club house / Community Center']  
2713                                                                                                                                                                                                                                                                          NaN  
130                                                                     ['Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Swimming Pool', 'Park', 'Security Personnel', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harvesting']  
1419                ['Private Garden / Terrace', 'High Ceiling Height', 'Maintenance Staff', 'Water Storage', 'Separate entry for servant room', 'No open drainage around', 'Park', 'Visitor Parking', 'Security Personnel', 'Natural Light', 'Airy Rooms', 'Spacious Interiors']  
2213                                                                                                                                                                                                                                                                          NaN

In [606]:
# Extract all unique furnishings from the furnishDetails column
all_furnishings = []
for detail in df['furnishDetails'].dropna():
    furnishings = detail.replace('[', '').replace(']', '').replace("'", "").split(', ')
    all_furnishings.extend(furnishings)
unique_furnishings = list(set(all_furnishings))

# Define a function to extract the count of a furnishing from the furnishDetails
def get_furnishing_count(details, furnishing):
    if isinstance(details, str):
        if f"No {furnishing}" in details:
            return 0
        pattern = re.compile(f"(\d+) {furnishing}")
        match = pattern.search(details)
        if match:
            return int(match.group(1))
        elif furnishing in details:
            return 1
    return 0

# Simplify the furnishings list by removing "No" prefix and numbers
columns_to_include = [re.sub(r'No |\d+', '', furnishing).strip() for furnishing in unique_furnishings]
columns_to_include = list(set(columns_to_include))  # Get unique furnishings
columns_to_include = [furnishing for furnishing in columns_to_include if furnishing]  # Remove empty strings

# Create new columns for each unique furnishing and populate with counts
for furnishing in columns_to_include:
    df[furnishing] = df['furnishDetails'].apply(lambda x: get_furnishing_count(x, furnishing))

# Create the new dataframe with the required columns
furnishings_df = df[['furnishDetails'] + columns_to_include]


In [607]:
furnishings_df.shape

(3803, 19)

In [608]:
furnishings_df.drop(columns=['furnishDetails'],inplace=True)

C:\Users\Nitish\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [610]:
furnishings_df.sample(5)

Exhaust Fan  Light  Chimney  Sofa  AC  Fan  Fridge  Dining Table  \
2110            1      1        0     0   0    2       0             0   
1549            0      0        0     0   0    0       0             0   
3035            1      0        0     0   0    0       0             0   
3031            1     14        1     0   0    6       0             0   
3535            0      0        0     0   0    0       0             0   

      Water Purifier  Modular Kitchen  Stove  Washing Machine  Geyser  \
2110               0                1      0                0       0   
1549               0                0      0                0       0   
3035               0                1      0                0       2   
3031               1                1      1                0       4   
3535               0                0      0                0       0   

      Microwave  Bed  Wardrobe  TV  Curtains  
2110          0    1         1   0         0  
1549          0    0         0   0         0  
3035          0    0         2   0         0  
3031          0    0         4   0         0  
3535          0    0         0   0         0

In [611]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [612]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(furnishings_df)

In [613]:
wcss_reduced = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(scaled_data)
    wcss_reduced.append(kmeans.inertia_)

C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\

In [614]:
# Plot the results
plt.figure(figsize=(12, 8))
plt.plot(range(1,11), wcss_reduced, marker='o', linestyle='--')
plt.title('Elbow Method For Optimal Number of Clusters (Reduced Range)')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.grid(True)
plt.show()

In [615]:
n_clusters = 3

# Fit the KMeans model
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(scaled_data)

# Predict the cluster assignments for each row
cluster_assignments = kmeans.predict(scaled_data)

C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [616]:
df = df.iloc[:,:-18]

In [617]:
df['furnishing_type'] = cluster_assignments

In [619]:
df.sample(5)[['furnishDetails','furnishing_type']]
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished

furnishDetails  \
3344                                                                                                                                                                                                                                                                 NaN   
3054  ['8 Wardrobe', '12 Fan', '4 Geyser', '12 AC', '4 Chimney', 'No Bed', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Modular Kitchen', 'No Light', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']   
3400  ['1 Fan', '1 Stove', '1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']   
3179      ['3 Wardrobe', '5 Fan', '1 Exhaust Fan', '3 Geyser', '10 Light', '1 Modular Kitchen', '1 Chimney', '1 Curtains', 'No AC', 'No Bed', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']   
3397  ['4 Wardrobe', '4 Fan', '1 Exhaust Fan', '10 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']   

      furnishing_type  
3344                0  
3054                1  
3400                0  
3179                1  
3397                0

## 5.features

In [628]:
df[['society','features']].sample(5)

society  \
3057                     indiabulls enigma   
1561                     dlf regal gardens   
2966                           ss the leaf   
1465  shapoorji pallonji joyville gurugram   
98               pivotal devaan apartments   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                features  
3057                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 NaN  
1561                                                                                                                                                                                                                                                                                                                                                                                                         ['Intercom Facility', 'Lift(s)', 'Swimming Pool', 'Park', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harvesting']  
2966  ['Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Centrally Air Conditioned', 'High Ceiling Height', 'Maintenance Staff', 'False Ceiling Lighting', 'Water Storage', 'Separate entry for servant room', 'No open drainage around', 'Visitor Parking', 'Swimming Pool', 'Park', 'Security Personnel', 'Natural Light', 'Internet/wi-fi connectivity', 'Airy Rooms', 'Low Density Society', 'Shopping Centre', 'Fitness Centre / GYM', 'Waste Disposal', 'Rain Water Harvesting', 'Club house / Community Center', 'Water softening plant']  
1465                                                                                                                                                                                                                                                               ['Feng Shui / Vaastu Compliant', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Piped-gas', 'Swimming Pool', 'Park', 'Security Personnel', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Rain Water Harvesting', 'Club house / Community Center']  
98                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   NaN

In [512]:
df['features'].isnull().sum()

635

In [513]:
import pandas as pd
app_df = pd.read_csv('appartments.csv')
app_df.head(2)

PropertyName                               PropertySubName  \
0  Smartworld One DXP  2, 3, 4 BHK Apartment in Sector 113, Gurgaon   
1           M3M Crown     3, 4 BHK Apartment in Sector 111, Gurgaon   

                                                                                                                                  NearbyLocations  \
0                                           ['Bajghera Road', 'Palam Vihar Halt', 'DPSG Palam Vihar', 'Park Hospital', 'Gurgaon Railway Station']   
1  ['DPSG Palam Vihar Gurugram', 'The NorthCap University', 'Park Hospital, Palam Vihar', 'Pacific D21 Mall', 'Palam Vihar Halt Railway Station']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    LocationAdvantages  \
0         {'Bajghera Road': '800 Meter', 'Palam Vihar Halt': '2.5 KM', 'DPSG Palam Vihar': '3.1 KM', 'Park Hospital': '3.1 KM', 'Gurgaon Railway Station': '4.9 KM', 'The NorthCap University': '5.4 KM', 'Dwarka Expy': '1.2 KM', 'Hyatt Place Gurgaon Udyog Vihar': '7.7 KM', 'Dwarka Sector 21, Metro Station': '7.2 KM', 'Pacific D21 Mall': '7.4 KM', 'Indira Gandhi International Airport': '14.7 KM', 'Hamoni Golf Camp': '6.2 KM', 'Fun N Food Waterpark': '8.8 KM', 'Accenture DDC5': '9 KM'}   
1  {'DPSG Palam Vihar Gurugram': '1.4 Km', 'The NorthCap University': '4.4 Km', 'Park Hospital, Palam Vihar': '1.4 Km', 'Pacific D21 Mall': '8.2 Km', 'Palam Vihar Halt Railway Station': '1.2 Km', 'Dwarka Sector 21 Metro Station': '8.1 Km', 'Dwarka Expressway': '450 m', 'Fun N Food Water Park': '8.1 Km', 'Indira Gandhi International Airport': '14.1 Km', 'Tau DeviLal Sports Complex': '11.2 Km', 'Hamoni Golf Camp': '5 Km', 'Hyatt Place': '6.1 Km', 'Altrade Business Centre': '11.2 Km'}   

                                                                          Link  \
0  https://www.99acres.com/smartworld-one-dxp-sector-113-gurgaon-npxid-r400415   
1           https://www.99acres.com/m3m-crown-sector-111-gurgaon-npxid-r404068   

                                                                                                                                                                                                                                                                                                                                                                                           PriceDetails  \
0  {'2 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,370 sq.ft.', 'price-range': '₹ 2 - 2.4 Cr'}, '3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,850 - 2,050 sq.ft.', 'price-range': '₹ 2.25 - 3.59 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,600 sq.ft.', 'price-range': '₹ 3.24 - 4.56 Cr'}}   
1                                                                                                       {'3 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '1,605 - 2,170 sq.ft.', 'price-range': '₹ 2.2 - 3.03 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '2,248 - 2,670 sq.ft.', 'price-range': '₹ 3.08 - 3.73 Cr'}}   

                                                                                                                                   TopFacilities  
0                     ['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']  
1  ['Bowling Alley', 'Mini Theatre', 'Manicured Garden', 'Swimming Pool', 'Flower Garden', 'Reading Lounge', 'Golf Course', 'Barbecue', 'Sauna']

In [514]:
app_df['PropertyName'] = app_df['PropertyName'].str.lower()

In [515]:
temp_df = df[df['features'].isnull()]

In [516]:
temp_df.shape

(635, 26)

In [631]:
x = temp_df.merge(app_df,left_on='society',right_on='PropertyName',how='left')['TopFacilities']

In [632]:
df.loc[temp_df.index,'features'] = x.values

In [633]:
df['features'].isnull().sum()

481

In [531]:
from sklearn.preprocessing import MultiLabelBinarizer
import ast

In [634]:
# Convert the string representation of lists in the 'features' column to actual lists
df['features_list'] = df['features'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) and x.startswith('[') else [])

# Use MultiLabelBinarizer to convert the features list into a binary matrix
mlb = MultiLabelBinarizer()
features_binary_matrix = mlb.fit_transform(df['features_list'])

# Convert the binary matrix into a DataFrame
features_binary_df = pd.DataFrame(features_binary_matrix, columns=mlb.classes_)

In [636]:
features_binary_df.sample(5)

24/7 Power Backup  24/7 Water Supply  24x7 Security  ATM  \
2490                  0                  0              0    0   
1122                  0                  0              0    0   
673                   0                  0              0    0   
642                   0                  0              0    0   
2266                  0                  0              0    0   

      Aerobics Centre  Air Hockey  Airy Rooms  Amphitheatre  \
2490                0           0           0             0   
1122                0           0           0             0   
673                 0           0           0             0   
642                 0           0           0             0   
2266                0           0           0             0   

      Automated Car Wash  Badminton Court  Bank Attached Property  \
2490                   0                0                       0   
1122                   0                0                       0   
673                    0                0                       0   
642                    0                0                       0   
2266                   0                0                       0   

      Banquet Hall  Bar/Chill-Out Lounge  Barbecue  Basketball Court  \
2490             0                     0         0                 0   
1122             0                     0         0                 0   
673              0                     0         0                 0   
642              0                     0         0                 0   
2266             0                     0         0                 0   

      Beach Volley Ball Court  Billiards  Bowling Alley  Bus Shelter  \
2490                        0          0              0            0   
1122                        0          0              0            0   
673                         0          0              0            0   
642                         0          0              0            0   
2266                        0          0              0            0   

      Business Lounge  CCTV Camera Security  Cafeteria  Car Parking  \
2490                0                     0          0            0   
1122                0                     0          0            0   
673                 0                     0          0            0   
642                 0                     0          0            0   
2266                0                     0          0            0   

      Car wash area  Card Room  Centrally Air Conditioned  Changing Area  \
2490              0          0                          0              0   
1122              0          0                          0              0   
673               0          0                          0              0   
642               0          0                          0              0   
2266              0          0                          0              0   

      Children's Play Area  Cigar Lounge  Clinic  Club House  \
2490                     0             0       0           0   
1122                     0             0       0           0   
673                      0             0       0           0   
642                      0             0       0           0   
2266                     0             0       0           0   

      Club house / Community Center  Community Hall  Concierge Service  \
2490                              1               0                  0   
1122                              0               0                  0   
673                               0               0                  0   
642                               0               0                  0   
2266                              0               0                  0   

      Conference room  Creche/Day care  Cricket Pitch  Doctor on Call  \
2490                0                0              0               0   
1122                0                0              0               0   
673             

In [543]:
features_binary_df.shape

(3803, 130)

In [535]:
wcss_reduced = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(features_binary_df)
    wcss_reduced.append(kmeans.inertia_)

C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Nitish\anaconda3\lib\site-packages\sklearn\

In [536]:
# Plot the results
plt.figure(figsize=(12, 8))
plt.plot(range(1,11), wcss_reduced, marker='o', linestyle='--')
plt.title('Elbow Method For Optimal Number of Clusters (Reduced Range)')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.grid(True)
plt.show()

In [544]:
# Define the weights for each feature as provided
# Assigning weights based on perceived luxury contribution
weights = {
    '24/7 Power Backup': 8,
    '24/7 Water Supply': 4,
    '24x7 Security': 7,
    'ATM': 4,
    'Aerobics Centre': 6,
    'Airy Rooms': 8,
    'Amphitheatre': 7,
    'Badminton Court': 7,
    'Banquet Hall': 8,
    'Bar/Chill-Out Lounge': 9,
    'Barbecue': 7,
    'Basketball Court': 7,
    'Billiards': 7,
    'Bowling Alley': 8,
    'Business Lounge': 9,
    'CCTV Camera Security': 8,
    'Cafeteria': 6,
    'Car Parking': 6,
    'Card Room': 6,
    'Centrally Air Conditioned': 9,
    'Changing Area': 6,
    "Children's Play Area": 7,
    'Cigar Lounge': 9,
    'Clinic': 5,
    'Club House': 9,
    'Concierge Service': 9,
    'Conference room': 8,
    'Creche/Day care': 7,
    'Cricket Pitch': 7,
    'Doctor on Call': 6,
    'Earthquake Resistant': 5,
    'Entrance Lobby': 7,
    'False Ceiling Lighting': 6,
    'Feng Shui / Vaastu Compliant': 5,
    'Fire Fighting Systems': 8,
    'Fitness Centre / GYM': 8,
    'Flower Garden': 7,
    'Food Court': 6,
    'Foosball': 5,
    'Football': 7,
    'Fountain': 7,
    'Gated Community': 7,
    'Golf Course': 10,
    'Grocery Shop': 6,
    'Gymnasium': 8,
    'High Ceiling Height': 8,
    'High Speed Elevators': 8,
    'Infinity Pool': 9,
    'Intercom Facility': 7,
    'Internal Street Lights': 6,
    'Internet/wi-fi connectivity': 7,
    'Jacuzzi': 9,
    'Jogging Track': 7,
    'Landscape Garden': 8,
    'Laundry': 6,
    'Lawn Tennis Court': 8,
    'Library': 8,
    'Lounge': 8,
    'Low Density Society': 7,
    'Maintenance Staff': 6,
    'Manicured Garden': 7,
    'Medical Centre': 5,
    'Milk Booth': 4,
    'Mini Theatre': 9,
    'Multipurpose Court': 7,
    'Multipurpose Hall': 7,
    'Natural Light': 8,
    'Natural Pond': 7,
    'Park': 8,
    'Party Lawn': 8,
    'Piped Gas': 7,
    'Pool Table': 7,
    'Power Back up Lift': 8,
    'Private Garden / Terrace': 9,
    'Property Staff': 7,
    'RO System': 7,
    'Rain Water Harvesting': 7,
    'Reading Lounge': 8,
    'Restaurant': 8,
    'Salon': 8,
    'Sauna': 9,
    'Security / Fire Alarm': 9,
    'Security Personnel': 9,
    'Separate entry for servant room': 8,
    'Sewage Treatment Plant': 6,
    'Shopping Centre': 7,
    'Skating Rink': 7,
    'Solar Lighting': 6,
    'Solar Water Heating': 7,
    'Spa': 9,
    'Spacious Interiors': 9,
    'Squash Court': 8,
    'Steam Room': 9,
    'Sun Deck': 8,
    'Swimming Pool': 8,
    'Temple': 5,
    'Theatre': 9,
    'Toddler Pool': 7,
    'Valet Parking': 9,
    'Video Door Security': 9,
    'Visitor Parking': 7,
    'Water Softener Plant': 7,
    'Water Storage': 7,
    'Water purifier': 7,
    'Yoga/Meditation Area': 7
}
# Calculate luxury score for each row
luxury_score = features_binary_df[list(weights.keys())].multiply(list(weights.values())).sum(axis=1)


In [545]:
df['luxury_score'] = luxury_score

In [546]:
df.head()

property_type                      society      sector  price  \
0          flat      signature global park 4   sector 36   0.82   
1          flat             smart world gems   sector 89   0.95   
2          flat                pyramid elite   sector 86   0.46   
3          flat       breez global hill view  sohna road   0.32   
4          flat  bestech park view sanskruti   sector 92   1.60   

   price_per_sqft     area  \
0          7585.0   1081.0   
1          8600.0   1105.0   
2            79.0  58228.0   
3          5470.0    585.0   
4          8020.0   1995.0   

                                                                                                             areaWithType  \
0                                             Super Built up area 1081(100.43 sq.m.)Carpet area: 650 sq.ft. (60.39 sq.m.)   
1                                                                                        Carpet area: 1103 (102.47 sq.m.)   
2                                                                                      Carpet area: 58141 (5401.48 sq.m.)   
3                                                   Built Up area: 1000 (92.9 sq.m.)Carpet area: 585 sq.ft. (54.35 sq.m.)   
4  Super Built up area 1995(185.34 sq.m.)Built Up area: 1615 sq.ft. (150.04 sq.m.)Carpet area: 1476 sq.ft. (137.12 sq.m.)   

   bedRoom  bathroom balcony           additionalRoom  floorNum      facing  \
0        3         2       2            not available       2.0         NaN   
1        2         2       2  study room,servant room       4.0         NaN   
2        2         2       1            not available       0.0         NaN   
3        2         2       1            not available      17.0         NaN   
4        3         4      3+      servant room,others      10.0  North-West   

        agePossession  \
0        New Property   
1        New Property   
2  Under Construction   
3        New Property   
4      Relatively New   

                                                                                                                                                                                                                                                                                                                                                nearbyLocations  \
0                                                    ['Sector 55-56 Metro Station', 'The Leaf Mall Sohna', 'Omaxe Gurgaon Mall', 'Badshahpur Sohna Rd Hwy', 'Vidya Niketan School', 'KIIT College of Engg', 'Scottish High International School', 'Rawal Institutions', 'Park Hospital', 'Indira Gandhi Intl Airport', 'Spaze Business Park', 'DLF Cyber City']   
1  ['Sant Soordas Sihi Metro Station', 'Airia Mall', 'Dwarka Expressway', 'Pataudi Road', 'NH-8', 'KMP Expressway', 'RPS International School', 'Genesis Hospital Sector 84', 'Gurgaon Railway Station', 'Vatika Business Park Sector 49', 'Holiday Inn Gurugram Sector 90', 'De Adventure Amusement Park', 'National Tennis Academy Sector 98', 'IMT Manesar']   
2                                                                                                                                                ['V3S Sapphire Ninety Mall', 'Sector 86 Road', 'Dwarka Expy', 'RPS International School', 'SGT University', 'Shree Radhey Hospital And Trauma Center', 'Indira Gandhi Intl Airport', 'Garhi Harsaru Junction']   
3                       ['Huda City Centre', 'Omaxe City Centre', 'Ninex City Mart', 'Vaels Ravindra Bharathi Global School', 'Anjali nursery', 'Polaris Hospital', 'Axis Bank', 'Triangular Park', 'Flow Sports Life Badminton Club', 'Basai Dhankot Railway Station', 'Cars24', 'Radisson Hotel Gurugram', "McDonald's", 'Petrol Pump', 'Indian Post Office']   
4                                   ['Sapphire 93 Mall', 'Dwarka Expressway', 'NH-8, Imt Manesar', 'Western Peripheral Expressway', 'RPS International School Sector 89', 'DPG Institute of Technology', 'Aarvy Healthcare Hospital', 'Indira Gandhi International Airport

In [550]:
# cols to drop -> nearbyLocations,furnishDetails, features,features_list, additionalRoom
df.drop(columns=['nearbyLocations','furnishDetails','features','features_list','additionalRoom'],inplace=True)

In [554]:
df.sample(5)

property_type                 society      sector  price  price_per_sqft  \
1254          flat             ss the leaf   sector 85   2.05          7290.0   
1978          flat  pioneer urban presidia   sector 62   6.50         14987.0   
203           flat    hsiidc sidco aravali     manesar   0.86          3323.0   
747          house             independent  sohna road   5.50          5093.0   
3467          flat  vatika the seven lamps   sector 82   0.94          6567.0   

         area  \
1254   2812.0   
1978   4337.0   
203    2588.0   
747   10799.0   
3467   1431.0   

                                                                                                                areaWithType  \
1254  Super Built up area 2812(261.24 sq.m.)Built Up area: 2500 sq.ft. (232.26 sq.m.)Carpet area: 2250 sq.ft. (209.03 sq.m.)   
1978                                                                                  Super Built up area 4337(402.92 sq.m.)   
203   Super Built up area 2588(240.43 sq.m.)Built Up area: 1900 sq.ft. (176.52 sq.m.)Carpet area: 1877 sq.ft. (174.38 sq.m.)   
747                                                Plot area 1200(1003.35 sq.m.)Built Up area: 3500 sq.yards (2926.45 sq.m.)   
3467                                                                                  Super Built up area 1435(133.32 sq.m.)   

      bedRoom  bathroom balcony  floorNum      facing   agePossession  \
1254        4         4      3+       4.0  North-East  Relatively New   
1978        4         4      3+       3.0        West  Moderately Old   
203         3         3      3+       3.0       South  Moderately Old   
747         3         3      3+       2.0        West  Moderately Old   
3467        2         2       2      12.0        East  Relatively New   

      super_built_up_area  built_up_area  carpet_area  study room  \
1254               2812.0         2500.0       2250.0           0   
1978               4337.0            NaN          NaN           1   
203                2588.0         1900.0       1877.0           0   
747                   NaN         3500.0          NaN           1   
3467               1435.0            NaN          NaN           1   

      servant room  store room  pooja room  others  furnishing_type  \
1254             1           0           0       0                1   
1978             0           0           0       0                0   
203              1           0           0       0                0   
747              1           0           1       1                0   
3467             0           0           0       0                1   

      luxury_score  
1254            49  
1978            86  
203             81  
747             49  
3467           103

In [552]:
df.shape

(3803, 23)

In [555]:
df.to_csv('gurgaon_properties_cleaned_v2.csv',index=False)